In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data1 = pd.read_csv('result/rsrp-l1800-alljabo-40x40-pop.csv/0.part')
data2 = pd.read_csv('result/rsrp-l1800-alljabo-40x40.csv/0.part')

existing_sites = pd.read_excel('sites/Existing Sites-L1800.xlsx')
new_sites = pd.read_csv('sites/1657 Sites.csv')

kecamatan = gpd.read_file('Polygon XL - EID Area/Kecamatan_Jabotabek.TAB')

In [3]:
data1.rename(columns={'rsrp-l1800':'rsrp-l1800-qty'}, inplace=True)

In [4]:
existing_sites = existing_sites[['Tower ID', 'Frequency System', 'Longitude', 'Latitude']]
new_sites = new_sites[['Planning ID','Lat', 'Long']]

In [5]:
data2.rename(columns={'geometry_polygon': 'geometry_polygon1'}, inplace=True)
data1 = pd.concat([data1, data2], axis=1, join='inner')

In [6]:
data1

,geometry_polygon,rsrp-l1800-qty,geometry_polygon1,rsrp-l1800
0,"POLYGON ((105.164179 -6.564111, 105.164538 -6....",1.0,"POLYGON ((105.164179 -6.564111, 105.164538 -6....",-78.000000
1,"POLYGON ((105.190774 -6.278029, 105.191134 -6....",1.0,"POLYGON ((105.190774 -6.278029, 105.191134 -6....",-93.000000
2,"POLYGON ((105.191493 -7.034925, 105.191852 -7....",1.0,"POLYGON ((105.191493 -7.034925, 105.191852 -7....",-85.000000
3,"POLYGON ((105.207307 -6.591425, 105.207666 -6....",65.0,"POLYGON ((105.207307 -6.591425, 105.207666 -6....",-97.369231
4,"POLYGON ((105.209463 -6.914885, 105.209822 -6....",1.0,"POLYGON ((105.209463 -6.914885, 105.209822 -6....",-124.000000
...,...,...,...,...
3953998,"POLYGON ((107.64332 -6.187819, 107.643679 -6.1...",1.0,"POLYGON ((107.64332 -6.187819, 107.643679 -6.1...",-96.000000
3953999,"POLYGON ((107.64332 -6.189976, 107.643679 -6.1...",1.0,"POLYGON ((107.64332 -6.189976, 107.643679 -6.1...",-95.000000
3954000,"POLYGON ((107.64332 -6.190335, 107.643679 -6.1...",2.0,"POLYGON ((107.64332 -6.190335, 107.643679 -6.1...",-92.500000
3954001,"POLYGON ((107.64332 -6.215134, 107.643679 -6.2...",1.0,"POLYGON ((107.64332 -6.215134, 107.643679 -6.2...",-115.000000


In [7]:
data1['compare'] = data1['geometry_polygon'] == data1['geometry_polygon1']
data1.reset_index(drop=True, inplace=True)
data1.drop(columns=['geometry_polygon1'], inplace=True)
data1 = data1[data1['rsrp-l1800-qty'] > 3000] 
data1 = data1[data1['rsrp-l1800'] < -105]

In [8]:
data1   = gpd.GeoDataFrame(data1, geometry=gpd.GeoSeries.from_wkt(data1['geometry_polygon']), crs=4326)
existing_sites = gpd.GeoDataFrame(existing_sites, geometry=gpd.points_from_xy(existing_sites['Longitude'], existing_sites['Latitude']), crs=4326)
new_sites = gpd.GeoDataFrame(new_sites, geometry=gpd.points_from_xy(new_sites['Long'], new_sites['Lat']), crs=4326)

In [9]:
data1 = data1.sjoin_nearest(existing_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_existing')
data1.drop(columns=['index_right'], inplace=True)
data1 = data1.sjoin_nearest(new_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_new')

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [10]:
data1.reset_index(inplace=True)

In [11]:
data1.drop(columns=['index'], inplace=True)

In [12]:
data1.rename(columns={'rsrp-l1800':'rsrp-l1800-avg'}, inplace=True)

In [13]:
data1.drop(columns=['index_right', 'geometry_polygon'], inplace=True)

In [14]:
data1 = gpd.sjoin(left_df= data1, right_df=kecamatan, how='left', predicate='within')

In [15]:
data1.to_csv('result/result-bad-grid-l1800.csv', index=False)